In [13]:
%matplotlib inline
import pandas as pd
import numpy as np
import math as math
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
data = pd.read_excel("data.xlsx")
data.head()

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Henry Cejudo,Marlon Moraes,Marc Goddard,2019,"Chicago, Illinois, USA",Red,True,Bantamweight,5,0,...,2,0,0,8,Orthodox,162.56,162.56,135.0,31.0,32.0
1,Valentina Shevchenko,Jessica Eye,Robert Madrigal,2019,"Chicago, Illinois, USA",Red,True,Women's Flyweight,5,0,...,0,2,0,5,Southpaw,165.10,167.64,125.0,32.0,31.0
2,Tony Ferguson,Donald Cerrone,Dan Miragliotta,2019,"Chicago, Illinois, USA",Red,False,Lightweight,3,0,...,3,6,1,14,Orthodox,180.34,193.04,155.0,36.0,35.0
3,Jimmie Rivera,Petr Yan,Kevin MacDonald,2019,"Chicago, Illinois, USA",Blue,False,Bantamweight,3,0,...,1,0,0,6,Orthodox,162.56,172.72,135.0,26.0,29.0
4,Tai Tuivasa,Blagoy Ivanov,Dan Miragliotta,2019,"Chicago, Illinois, USA",Blue,False,Heavyweight,3,0,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0


### Apenas a categoria Peso-Pesados ('Heavyweights')

In [15]:
data.weight_class = data.weight_class.astype('category')
data.Winner = data.Winner.astype('category')

In [16]:
data.Winner.value_counts()

Red     3470
Blue    1591
Draw      83
Name: Winner, dtype: int64

### Blue = 0
### Red = 1

In [17]:
data_heavy = data.loc[(data.weight_class=='Heavyweight'),:]
bool_to_number = {False: 0, True: 1}
string_to_number = {'Blue': 0, 'Red': 1, 'Draw': 2}
data_heavy['title_bout'] = data_heavy['title_bout'].map(bool_to_number)
data_heavy['Winner'] = data_heavy['Winner'].map(string_to_number)

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [18]:
data_heavy_util = data_heavy.drop(['Referee','date','location'], axis=1)

In [19]:
data_heavy_util.head()

,R_fighter,B_fighter,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,0,0,Heavyweight,3,0,1,0,17.0,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
54,Walt Harris,Serghei Spivac,1,0,Heavyweight,3,0,0,0,NaN,...,4,0,0,4,Southpaw,195.58,195.58,250.0,24.0,35.0
60,Arjan Bhullar,Juan Adams,1,0,Heavyweight,3,0,1,0,8.0,...,0,0,0,2,Orthodox,185.42,190.50,245.0,27.0,32.0
63,Greg Hardy,Dmitrii Smoliakov,1,0,Heavyweight,3,2,0,0,5.0,...,0,0,0,0,Orthodox,195.58,203.20,265.0,36.0,30.0
69,Andrei Arlovski,Augusto Sakai,0,0,Heavyweight,3,0,1,0,35.0,...,9,2,0,16,Orthodox,190.50,195.58,240.0,27.0,40.0


In [20]:
colunas = list(data_heavy_util)

In [21]:
colunas.remove('R_fighter')
colunas.remove('B_fighter')
colunas.remove('title_bout')
colunas.remove('weight_class')
colunas.remove('no_of_rounds')
colunas.remove('B_total_rounds_fought')
colunas.remove('R_total_rounds_fought')


In [22]:
dataf = data_heavy_util.loc[:,colunas]
dataf.head()

,Winner,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,0,0,1,0,17.0,14.5,2.5,2.0,201.0,59.5,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
54,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,4,0,0,4,Southpaw,195.58,195.58,250.0,24.0,35.0
60,1,0,1,0,8.0,8.0,15.0,13.0,85.0,38.0,...,0,0,0,2,Orthodox,185.42,190.50,245.0,27.0,32.0
63,1,2,0,0,5.0,3.5,5.5,3.5,16.0,5.5,...,0,0,0,0,Orthodox,195.58,203.20,265.0,36.0,30.0
69,0,0,1,0,35.0,21.0,38.0,30.0,167.0,69.0,...,9,2,0,16,Orthodox,190.50,195.58,240.0,27.0,40.0


In [23]:
x=colunas
y=x


In [ ]:
#sns.set(style="ticks")
#sns.pairplot(data_heavy_util, hue="origin", x_vars=x, y_vars=y)

KeyboardInterrupt: 

### Correlações

In [ ]:
data_heavy_blue_winners = data_heavy.loc[(data_heavy.Winner==0),:]
data_heavy_blue_winners = data_heavy_blue_winners.drop(['Winner'], axis=1)
data_heavy_red_winners = data_heavy.loc[(data_heavy.Winner==1),:]
data_heavy_red_winners = data_heavy_red_winners.drop(['Winner'], axis=1)

In [ ]:
blue_winners_correlation = data_heavy_blue_winners.corr()

### Aplicando filtros

In [ ]:
filtros = ['R_fighter','B_fighter','date','Winner','weight_class','title_bout','R_wins','R_losses','B_wins','B_losses',
           'B_Reach_cms','B_Height_cms','B_Weight_lbs','R_Reach_cms','R_Height_cms','R_Weight_lbs']

In [ ]:
data_heavy_filtered = data_heavy.loc[:,filtros]

In [ ]:
data_heavy_filtered